# Initial Setup

## 📬 Setting Up Gmail API Access

To enable your AI agent to read and send emails, you'll need to set up the Gmail API on your Google account. Follow these steps carefully:

- 🔑 Step 1: Open Google Cloud Console
  - Go to [https://console.cloud.google.com/](https://console.cloud.google.com/)
  - Sign in using the **Google account** you want the agent to access

- 🆕 Step 2: Create a New Project
  - Click the project dropdown (top nav bar)
  - Click **"New Project"**, give it a name, and click **"Create"**

- ⚙️ Step 3: Enable Gmail API
  - Navigate to **APIs & Services > Library**
  - Search for **Gmail API**
  - Click on it, then click **"Enable"**

- 🔐 Step 4: Configure OAuth Consent Screen
  - Go to **APIs & Services > OAuth consent screen**
  - Choose **"External"**
  - Fill in required fields (app name, user support email, etc.)
  - Add Gmail API scopes:

- 🧾 Step 5: Create OAuth 2.0 Credentials
  - Go to **APIs & Services > Credentials**
  - Click **"Create Credentials" → "OAuth Client ID"**
  - Choose **Desktop App**
  - Download the `credentials.json` file
  - Store the json file at `/content/drive/MyDrive/email_assistant/`

🔁 Save this file in the same directory as your notebook. We'll use it to authenticate Gmail access.

## 💡 Setting Up Gemini API Access (Google AI Studio)

To generate intelligent email replies, we'll use Google's Gemini model via the Generative Language API.

- 🔗 Step 1: Visit Google AI Studio
  - Go to [https://ai.google.dev/](https://ai.google.dev/)
  - Sign in with your Google account

- 📋 Step 2: Create a New API Key
  - Click on **"Get API Key"**
  - Accept the terms and select your project
  - Copy the generated **API key**

- 🗝️ Step 3: Paste the API Key
  - You’ll need to `paste the Gemini API Key` when prompted

# Dependancies

## Install Required Libraries

- 📦 Installing Required Python Libraries
    - `google-api-python-client`, `google-auth-httplib2`, `google-auth-oauthlib`: For Gmail API authentication and access
    - `google-generativeai`: Gemini LLM SDK from Google AI
    - `pymupdf`: For reading PDF attachments
    - `python-docx`: For reading DOCX attachments

✅ Run this cell once to set up your environment. If you're running in Colab or a fresh environment, this step is required.

In [ ]:
# 📬 Gmail API libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# 🤖 Gemini + file parsing libraries
!pip install --upgrade google-generativeai pymupdf python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.164.0
    Uninstalling google-api-python-client-2.164.0:
      Successfully uninstalled google-api-python-client-2.164.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.0 MB/s eta 0:00:00


## Mounting Google Drive

This step mounts your Google Drive to the Colab environment so you can:

- Load sensitive files like `credentials.json` from a secure location
- Save your `token.pickle` after authenticating with Gmail
- Store logs, results, or exports if needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Required Libraries

This block imports all required libraries for:

- 📬 Gmail authentication and API access
- 🧠 Gemini LLM integration
- 📎 File handling (PDF, DOCX, CSV, XLSX, base64 decoding)
- 🛠️ Utility functions like regex, token handling, and retry logic

In [ ]:
# 🌐 File and system utilities
import os
import pickle
import json
import re
import base64
import textwrap
import time
import random

# 📬 Gmail API
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from google.api_core.exceptions import TooManyRequests  # For rate limit handling

# 📎 File readers
import fitz  # PyMuPDF for reading PDFs
import docx  # python-docx for Word files
import csv   # For CSV files
import pandas as pd  # For Excel (XLSX) files

# 🔐 Secure API key input
import getpass

# 🤖 Gemini LLM
import google.generativeai as genai

# 📬 Gmail Client Class

This class handles all interactions with the Gmail API including:

- Authenticating the user via OAuth 2.0 (using `credentials.json`)
- Reading `unread emails` and extracting content
- Downloading attachments
- `Sending emails` with MIME formatting
- `Marking emails as read` after processing

🔒 Note: Once authenticated, credentials are cached using a `token.pickel` file to avoid repeat authorization prompts.


In [ ]:
class GmailClient:
    def __init__(self, creds_path, token_path, save_dir="/content"):
        self.creds_path = creds_path
        self.token_path = token_path
        self.save_dir = save_dir
        self.service = self.authenticate()      # Authenticate and build Gmail API service
        os.makedirs(save_dir, exist_ok=True)    # Ensure attachment save directory exists

    def authenticate(self):
        # ✅ Reuse token if already authenticated
        if os.path.exists(self.token_path):
            with open(self.token_path, 'rb') as token:
                creds = pickle.load(token)
        else:
            # 🔐 OAuth flow for first-time login
            flow = Flow.from_client_secrets_file(
                self.creds_path,
                scopes=["https://www.googleapis.com/auth/gmail.modify"],
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )
            auth_url, _ = flow.authorization_url(prompt='consent')
            print("🔗 Go to this URL, authorize, and paste the code below:")
            print(auth_url)
            code = input("Paste your code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

            # 💾 Save token for future sessions
            with open(self.token_path, 'wb') as token:
                pickle.dump(creds, token)

        # ✅ Build Gmail API service client
        return build('gmail', 'v1', credentials=creds)

    def get_unread_emails(self, max_results=10):
        # 📥 Get unread emails from inbox with metadata and decoded body
        response = self.service.users().messages().list(
            userId='me', labelIds=['INBOX'], q='is:unread').execute()
        messages = response.get('messages', [])[:max_results]

        emails = []
        for msg in messages:
            msg_data = self.service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
            payload = msg_data.get('payload', {})
            headers = payload.get('headers', [])

            # 📩 Extract subject and sender from headers
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'No Subject')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown Sender')

            # 📜 Extract plain-text body from payload (decode base64)
            parts = payload.get('parts', [])
            data = payload.get('body', {}).get('data', '')
            if parts and 'data' in parts[0]['body']:
                data = parts[0]['body']['data']
            try:
                decoded_body = base64.urlsafe_b64decode(data.encode('UTF-8')).decode('utf-8', errors='ignore')
            except Exception:
                decoded_body = "[Error decoding body]"

            emails.append({
                'id': msg['id'],
                'subject': subject,
                'from': sender,
                'body': decoded_body.strip()
            })
        return emails

    def get_attachments(self, msg_id):
        # 📎 Download attachments from a message and return file paths
        attachments_info = []
        msg = self.service.users().messages().get(userId='me', id=msg_id).execute()
        parts = msg.get('payload', {}).get('parts', [])

        for part in parts:
            if part.get('filename') and 'attachmentId' in part.get('body', {}):
                # 🧷 Download and save the attachment
                attachment_id = part['body']['attachmentId']
                attachment = self.service.users().messages().attachments().get(
                    userId='me', messageId=msg_id, id=attachment_id).execute()
                file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                file_path = os.path.join(self.save_dir, part['filename'])

                # 💾 Save to local disk
                with open(file_path, 'wb') as f:
                    f.write(file_data)
                attachments_info.append(file_path)

        return attachments_info

    def create_message(self, to, subject, message_text):
        # 📨 Create base64-encoded MIME message
        message = MIMEText(message_text)
        message['to'] = to
        message['subject'] = subject
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        return {'raw': raw}

    def send_email(self, to, subject, body):
        # 📤 Send email using Gmail API
        message = self.create_message(to, subject, body)
        sent = self.service.users().messages().send(userId='me', body=message).execute()
        print(f"✅ Email sent to {to}. Message ID: {sent['id']}")

    def mark_as_read(self, msg_id):
        # ✅ Remove the 'UNREAD' label to mark message as processed
        self.service.users().messages().modify(
            userId='me', id=msg_id, body={'removeLabelIds': ['UNREAD']}
        ).execute()

# 📎 File Processor Class

This class handles the extraction of readable text from various file formats attached to emails.

## Supported File Types:
- PDF (.pdf)
- Word Document (.docx)
- Plain Text (.txt)
- CSV (.csv)
- Excel (.xlsx)

⚠️ If the file type is not supported or cannot be opened, a descriptive error string is returned.

In [ ]:
class FileProcessor:
    def __init__(self):
        pass

    def extract_text(self, file_path):
        # 🔍 Determine file type and delegate to the correct method
        if file_path.endswith(".pdf"):
            return self._extract_text_from_pdf(file_path)
        elif file_path.endswith(".docx"):
            return self._extract_text_from_docx(file_path)
        elif file_path.endswith(".txt"):
            return self._extract_text_from_txt(file_path)
        elif file_path.endswith(".csv"):
            return self._extract_text_from_csv(file_path)
        elif file_path.endswith(".xlsx"):
            return self._extract_text_from_xlsx(file_path)
        else:
            return f"[Unsupported file type: {file_path}]"

    def _extract_text_from_pdf(self, file_path):
        # 📄 Read and extract text from PDF using PyMuPDF
        try:
            doc = fitz.open(file_path)
            return "\n".join([page.get_text() for page in doc])
        except Exception as e:
            return f"[Error reading PDF: {e}]"

    def _extract_text_from_docx(self, file_path):
        # 📄 Read and extract paragraphs from Word documents
        try:
            doc = docx.Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        except Exception as e:
            return f"[Error reading DOCX: {e}]"

    def _extract_text_from_txt(self, file_path):
        # 📄 Read plain text from .txt files
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
        except Exception as e:
            return f"[Error reading TXT: {e}]"

    def _extract_text_from_csv(self, file_path):
        # 📄 Read rows from CSV and join as lines
        try:
            lines = []
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                reader = csv.reader(f)
                for row in reader:
                    lines.append(", ".join(row))
            return "\n".join(lines)
        except Exception as e:
            return f"[Error reading CSV: {e}]"

    def _extract_text_from_xlsx(self, file_path):
        # 📄 Read all sheets from Excel and format as string
        try:
            df = pd.read_excel(file_path, sheet_name=None)
            content = []
            for sheet, data in df.items():
                content.append(f"Sheet: {sheet}")
                content.append(data.to_string(index=False))
            return "\n\n".join(content)
        except Exception as e:
            return f"[Error reading XLSX: {e}]"

# 🤖 LLMResponder Class

This class is responsible for interacting with the Gemini LLM to:
- Generate email summaries
- Classify sentiment
- Suggest intelligent replies
- Rank emails by urgency/importance

It also includes built-in retry logic to gracefully handle Gemini API rate limits.

In [ ]:
class LLMResponder:
    def __init__(self, model, signature="Best,\nRohit Akole\nData Analyst | AI Enthusiast"):
        self.model = model
        self.signature = signature

    def _retry_generate(self, prompt, max_attempts=3):
        # 🔁 Retry mechanism for rate-limited LLM requests
        for attempt in range(max_attempts):
            try:
                return self.model.generate_content(prompt).text.strip()
            except TooManyRequests:
                wait = 2 ** attempt + random.uniform(0, 1)
                print(f"⚠️ Rate limited. Retrying in {wait:.2f}s...")
                time.sleep(wait)
        raise Exception("Rate limit exceeded")

    def detect_sentiment(self, text):
        # 🧠 Predict emotional tone of the input text
        prompt = (
            "Analyze the emotional tone of the message below. Choose the most fitting emotion from "
            "the following list: Joy, Anger, Disappointment, Anxiety, Frustration, Sadness, Neutral, Funny. "
            "Return only the matching emotion word.\n\n"
            f"{text.strip()}"
        )
        response = self._retry_generate(prompt).lower()
        emoji_map = {
            "joy": "🙂 Joy", "anger": "😠 Anger", "disappointment": "😞 Disappointment",
            "anxiety": "😰 Anxiety", "frustration": "😡 Frustration", "sadness": "😢 Sadness",
            "neutral": "😐 Neutral", "funny": "😂 Funny"
        }
        for keyword, emoji in emoji_map.items():
            if keyword in response:
                return emoji
        return "😐 Neutral"  # fallback if tone unclear

    def summarize_text(self, text, prefix="Summarize this text:"):
        # 📄 Generate a summary from the input text (limited to ~3000 chars)
        prompt = f"{prefix}\n\n{text.strip()[:3000]}"
        return self._retry_generate(prompt) or "No summary available."

    def summarize_email_with_attachments(self, email_body, attachments_text):
        # 🧾 Combine email and attachments for a unified summary
        combined_text = (email_body.strip() + "\n" + attachments_text.strip()).strip()
        sentiment = self.detect_sentiment(combined_text)
        body_summary = self.summarize_text(email_body, prefix="Summarize this email:")

        attachment_summary = ""
        if attachments_text:
            attachment_summary = self.summarize_text(attachments_text, prefix="Summarize the attached content and give important bullets:")

        return f"{body_summary}\n\n{attachment_summary}\n\n{sentiment}".strip()

    def generate_reply_options(self, email_body, num_options=3):
        # ✉️ Generate multiple reply options, each with a subject and body
        prompt = (
            f"The following is an email that needs a reply:\n\n{email_body.strip()}\n\n"
            f"Generate {num_options} professional, friendly email replies."
            f"If the sender's name is identifiable, use it in a friendly greeting."
            f"Otherwise, omit the greeting without using a placeholder."
            f"End each reply with this signature:\n\n{self.signature}, can you also add 1 empty line between body and signature.\n"
            f"Format each option like:\n\nSubject: [Your subject here]\nBody: [Your reply here]"
        )

        raw_output = self._retry_generate(prompt)
        blocks = raw_output.split("Subject:")[1:]  # Split replies by Subject marker

        options = []
        for block in blocks:
            lines = block.strip().splitlines()
            subject = lines[0].strip()
            remaining = "\n".join(lines[1:]).strip()
            if remaining.lower().startswith("body:"):
                remaining = remaining[5:].strip()

            # ✂️ Truncate anything after the signature
            if self.signature in remaining:
                body = remaining.split(self.signature)[0].strip() + f"\n{self.signature}"
            else:
                body = remaining.strip()

            options.append({"subject": subject, "body": body})
        return options

    def rank_emails_by_importance(self, email_summaries):
        # 📊 Ask LLM to rank emails based on perceived urgency and relevance
        prompt = "You are an email assistant. Rank the emails by urgency and importance.\n\n"
        for i, email in enumerate(email_summaries, 1):
            prompt += f"{i}. From: {email['from']}, Subject: {email['subject']}\nSummary: {email['summary']}\n\n"
        prompt += "Return a comma-separated list like: 2,1,3"

        response = self._retry_generate(prompt)
        order = [int(x.strip()) - 1 for x in response.strip().split(',') if x.strip().isdigit()]
        return [i for i in order if 0 <= i < len(email_summaries)]

# 🔧 replace_placeholders() Utility

This function scans the input text for placeholders in the format `[placeholder]`:
- Replaces them using `predefined_values` if provided
- Prompts the user interactively for missing values
- Skips over optional fields like `[your name]`, `[title]`, etc.

This ensures no placeholder is left in the final email before sending.


In [ ]:
def replace_placeholders(text, predefined_values=None):
    # 🔄 Replace placeholders like [name], [project], [your name] in the email body
    def clean_input(val):
        # ✂️ Remove outer brackets if user types them
        val = val.strip()
        while val.startswith("[") and val.endswith("]"):
            val = val[1:-1].strip()
        return val

    # 🧠 Find all [placeholders] using regex
    placeholders = re.findall(r"\[([^\[\]]{1,80})\]", text)
    filled = {}

    for ph in placeholders:
        if ph in filled:
            continue

        key = ph.lower().strip()

        # ✅ Use predefined value if available
        if predefined_values and key in predefined_values:
            filled[ph] = predefined_values[key]

        # 🚫 Auto-skip common optional fields
        elif "your name" in key or "title" in key or "contact" in key:
            filled[ph] = ""

        # ✏️ Ask user for value
        else:
            print(f"✏️ What should replace '{ph}'?\n")
            filled[ph] = clean_input(input("→ ").strip())

    # 🔁 Replace all instances in the text
    for original, repl in filled.items():
        text = text.replace(f"[{original}]", repl)
        text = text.replace(f"[{original.upper()}]", repl)

    return text

# 🧼 clean_extra_gemini_comments() Utility

Removes unnecessary explanations or markdown-style notes from Gemini's output, such as:

- "**Explanation**"
- "**Key Considerations**"
- "To get the *best* results..."
- "Please note:"
- And other LLM-generated helper text

Ensures that the final email is polished and human-ready.


In [ ]:
def clean_extra_gemini_comments(text):
    # 📌 Markers that often indicate start of unnecessary LLM-generated commentary
    markers = [
        "**Explanation**", "**Option", "**Key Considerations", "**Important Considerations",
        "To get the *best*", "Please note:", "Here’s how I would approach this", "Some notes:"
    ]

    # ✂️ Remove everything after the first matching marker
    for marker in markers:
        if marker in text:
            text = text.split(marker)[0].strip()
    return text.strip()

# 🧠 Email Agent Controller Class

The core logic controller that ties everything together.

This class:
- Retrieves unread emails via Gmail API
- Summarizes content (including attachments)
- Detects tone/sentiment using Gemini
- Generates multiple AI-driven reply suggestions
- Prompts the user to review/edit/send replies
- Sends selected emails and marks them as read

🤝 This class puts the **human in control** while still leveraging AI to speed up reply generation.

In [ ]:
class EmailAgent:
    def __init__(self, gmail_client, llm_responder, file_processor):
        # 💡 Inject dependencies: Gmail, LLM, File handling
        self.gmail = gmail_client
        self.llm = llm_responder
        self.files = file_processor

    def wrapped_summary(self, text, width=100):
        # 📏 Print wrapped text (used for summaries and emails)
        for paragraph in text.strip().split('\n'):
            print(textwrap.fill(paragraph, width=width))

    def prepare_email_summaries(self, emails):
        # 🧾 Enrich emails with LLM-powered summaries and sentiment
        summaries = []
        for email in emails:
            # Download files
            attachments = self.gmail.get_attachments(email['id'])
            # Extract content
            attachment_text = "\n".join([self.files.extract_text(f) for f in attachments])
            full_summary = self.llm.summarize_email_with_attachments(email['body'], attachment_text)

            summaries.append({
                'id': email['id'],
                'from': email['from'],
                'subject': email['subject'],
                'body': email['body'],
                'summary': full_summary
            })
        return summaries

    def reply_to_emails(self):
        # 🔁 Loop through unread emails and allow replies
        while True:
            emails = self.gmail.get_unread_emails()
            if not emails:
                print("📭 No unread emails to reply to.")
                break

            email_summaries = self.prepare_email_summaries(emails)
            email_order = self.llm.rank_emails_by_importance(email_summaries)

            for idx in email_order:
                current_email = email_summaries[idx]
                print(f"\n📨 From: {current_email['from']}")
                print(f"🧾 Subject: {current_email['subject']}")
                print("📝 Summary:\n")
                self.wrapped_summary(current_email['summary'])

                # 🚦 Prompt to reply/skip/exit
                print("\nReply to this email? ['YES / Y', 'SKIP / S', 'EXIT / E']:\n")
                action = input("→ ").strip().lower()

                if action in ['skip', 's']:
                    continue
                elif action in ['exit', 'e']:
                    return
                elif action not in ['yes', 'y']:
                    print("❌ Invalid input. Skipping.")
                    continue

                # 📜 Use either full email or fallback to summary
                base_text = current_email['body'] if len(current_email['body']) > 50 else current_email['summary']

                while True:
                    # ✉️ Generate reply suggestions
                    reply_options = self.llm.generate_reply_options(base_text)
                    if not reply_options:
                        print("⚠️ No replies generated.")
                        break

                    # 📋 Display options
                    for i, opt in enumerate(reply_options, 1):
                        print(f"\nOption {i}")
                        print(f"Subject: {opt['subject']}")
                        print("Body:")
                        self.wrapped_summary(opt['body'])

                    # 🛠️ Allow user to select or edit a response
                    print(f"\nChoose reply (1-{len(reply_options)}), 'EDIT / ED', 'MORE / M', or 'EXIT / E':")
                    choice = input("→ ").strip().lower()

                    if choice.isdigit() and 1 <= int(choice) <= len(reply_options):
                        subject = reply_options[int(choice) - 1]['subject']
                        body = reply_options[int(choice) - 1]['body']
                        break
                    elif choice in ['edit', 'ed']:
                        print("✏️ Subject:")
                        subject = input("→ ").strip()
                        print("📝 Body (end with 'DONE'):")
                        lines = []
                        while True:
                            line = input()
                            if line.strip().lower() == 'done':
                                break
                            lines.append(line)
                        body = "\n".join(lines).strip()
                        break
                    elif choice in ['more', 'm']:
                        continue
                    elif choice in ['exit', 'e']:
                        return
                    else:
                        print("❌ Invalid. Try again.")

                # 🧼 Final cleanup before sending
                body = clean_extra_gemini_comments(body)
                body = re.sub(r"\n?\*\*Option.*?\*\*", "", body, flags=re.IGNORECASE).strip()
                body = replace_placeholders(body)

                # 👀 Final preview
                print("\n📤 Final Preview:\n")
                print(f"Subject: {subject}\nBody:\n")
                self.wrapped_summary(body)

                # ✅ Confirm before sending
                print("\nSend this email? ['YES / Y', 'NO / N']:")
                confirm = input("→ ").strip().lower()

                if confirm in ['yes', 'y']:
                    self.gmail.send_email(current_email['from'], subject, body)
                    self.gmail.mark_as_read(current_email['id'])
                    print("✅ Sent and marked as read.")
                else:
                    print("🚫 Email not sent.")

                # ⏱️ Avoid hitting rate limit
                time.sleep(4)

# 🔐 Configuring Gemini API Access

This cell securely prompts the user for their **Gemini API key** and uses it to configure the `google.generativeai` SDK. This is necessary for generating intelligent email replies via Google's large language models.

🛡️ Your API key is not displayed in plain text for security purposes. It must have access to the Generative Language API on [https://ai.google.dev](https://ai.google.dev).


In [ ]:
# 🔐 Prompt user securely for Gemini API key (input hidden)
gemini_api_key = getpass.getpass("🔐 Enter your Gemini API key: ")

# ⚙️ Configure the Gemini client using the provided key
genai.configure(api_key=gemini_api_key)

🔐 Enter your Gemini API key: ··········


# ⚡ Initializing Gemini Model

This cell initializes the Gemini model that will be used to generate intelligent email replies.

We're using:
- `gemini-2.0-flash` - a fast, lightweight version ideal for real-time applications

You can also switch to `gemini-pro` if you want more nuanced replies at the cost of speed.

In [ ]:
# ⚙️ Instantiate Components
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

# ✉️ Initialize Gmail Client

This cell creates an instance of the `GmailClient` class, which handles:

- Gmail API authentication using your `credentials.json` and `token.pkl` files
- Fetching **unread emails, sending replies,** and **marking messages as read**

🔁 If this is your first time running the notebook, you'll be prompted to authorize access to your Gmail account.

In [ ]:
# ✉️ Initialize Gmail client with credentials and token from Google Drive
gmail = GmailClient(
    creds_path='/content/drive/MyDrive/email_assistant/credentials.json',
    token_path='/content/drive/MyDrive/email_assistant/token.pkl'
)

# 🤖 Create and Run the AI Email Agent

Initialize all components and start the interactive loop to:

- Read unread emails
- Summarize content + attachments
- Detect sentiment
- Suggest intelligent replies
- Let the user choose, edit, or skip replies
- Send confirmed responses and mark emails as read

🟢 Run this cell to start your AI-powered email assistant!


In [ ]:
# 🧠 Instantiate the LLM responder using Gemini model
responder = LLMResponder(model)

# 📎 Instantiate file handler for parsing attachments
file_processor = FileProcessor()

# 🤖 Instantiate the main agent controller with Gmail, LLM, and file support
agent = EmailAgent(gmail, responder, file_processor)

# 🚀 Start the email agent workflow
agent.reply_to_emails()


📨 From: Google <no-reply@accounts.google.com>
🧾 Subject: Security alert
📝 Summary:

This email is a security notification from Google informing you that Google Drive for desktop was
granted access to your Google account (aidaytestemail@gmail.com). If you did not authorize this,
Google urges you to check the activity and secure your account using the provided links.



😰 Anxiety

Reply to this email? ['YES / Y', 'SKIP / S', 'EXIT / E']:

→ s

📨 From: Ananya Shrivastava <ananya.shrivastava.98@gmail.com>
🧾 Subject: your vaccum cleaner
📝 Summary:

Please provide me with the email you would like me to summarize. I need the text of the email to be
able to summarize it for you.

This user is extremely dissatisfied with their Bissell vacuum cleaner. They claim it won't turn on,
is too heavy to carry, and that it somehow trapped their three-month-old kitten inside.

**Key Complaints:**

*   **Non-functional:** Vacuum cleaner won't turn on.
*   **Excessively Heavy:** Too heavy to lift and move.

# 🌟 Real-World Use Cases
1. **Customer Support Auto-Replies**\
Summarize support requests, detect emotion (e.g. frustration), and draft appropriate replies or escalation emails.

2. **Internal HR Queries**\
Handle questions related to leave, payroll, or policies by reading employee emails and attaching relevant policies automatically.

3. **Sales Prospect Follow-ups**\
Read leads' emails, understand interest level, generate tiered responses (neutral, excited, ask for call), and auto-reply with signature.

4. **Executive Assistants for Busy Professionals**\
Filter unread inbox by priority, summarize key points from attachments, and suggest fast responses — like having a personal AI assistant.

# 🚀 Next-Level Feature Ideas (Future Enhancements)

1. **📆 Smart Scheduling Integration**  
   Automatically extract and schedule meetings or office hours into Google Calendar based on email content.

2. **📊 Email Dashboard Analytics**  
   Visualize trends such as common student concerns, inquiry volume by topic, sentiment shifts, and follow-up rates.

3. **🧠 Personalized Reply Memory**  
   Remember your past replies and preferences to generate consistent and context-aware responses.

4. **📁 Attachment Auto-Saving**  
   Automatically store received syllabi, reports, or proposals into a categorized Google Drive folder.

5. **🔍 Contextual Email Search with AI**  
   Search through emails using intent (e.g., “Find emails where students were dissatisfied”) instead of just keywords.

6. **🛡️ Phishing or Spam Detection:**  
  Analyze language and sender patterns to flag potential phishing or spam emails using NLP.